### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5524,2023-11-04,Espanha Acb,14:00,Zaragoza,Bilbao,1.59,2.50,164.5,1.95,1.87,-3.5,1.91,1.91
5525,2023-11-04,Espanha Acb,16:45,Tenerife,MoraBanc Andorra,1.33,3.40,163.5,1.91,1.91,-7.5,1.95,1.87
5526,2023-11-04,Eua Nba,20:00,Orlando Magic,Los Angeles Lakers,2.40,1.63,218.5,1.91,1.91,3.5,1.91,1.91
5527,2023-11-04,Eua Nba,21:00,Brooklyn Nets,Boston Celtics,4.50,1.22,229.5,1.91,1.91,9.5,1.91,1.91
5528,2023-11-04,Eua Nba,21:00,Minnesota Timberwolves,Utah Jazz,1.33,3.40,225.5,1.91,1.91,-7.5,1.91,1.91
5529,2023-11-04,Itália Liga A,16:30,Pesaro,Basket Napoli,2.05,1.80,163.5,1.91,1.91,-2.5,1.91,1.91
5530,2023-11-04,Alemanha Bbl,14:30,Syntainics MBC,Gottingen,1.80,2.05,165.5,1.91,1.91,-1.5,1.91,1.91
5531,2023-11-04,Alemanha Bbl,16:00,Crailsheim Merlins,Oldenburg,3.20,1.37,168.5,1.91,1.91,6.5,1.91,1.91
5532,2023-11-04,Alemanha Bbl,16:00,Wurzburg,Chemnitz,1.80,2.05,157.5,1.91,1.91,-1.5,1.91,1.91
5533,2023-11-04,Austrália Nbl,06:00,Perth Wildcats,Adelaide 36ers,1.42,2.90,172.5,1.91,1.91,-5.5,1.91,1.91


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Espanha Acb,Zaragoza,Bilbao,164.5,1.95,0.7651,Over
1,16:45,Espanha Acb,Tenerife,MoraBanc Andorra,163.5,1.91,0.7361,Over
2,20:00,Eua Nba,Orlando Magic,Los Angeles Lakers,218.5,1.91,0.7743,Over
3,21:00,Eua Nba,Brooklyn Nets,Boston Celtics,229.5,1.91,0.7916,Over
4,21:00,Eua Nba,Minnesota Timberwolves,Utah Jazz,225.5,1.91,0.7965,Over
5,16:30,Itália Liga A,Pesaro,Basket Napoli,163.5,1.91,0.8561,Over
6,14:30,Alemanha Bbl,Syntainics MBC,Gottingen,165.5,1.91,0.8380,Over
7,16:00,Alemanha Bbl,Crailsheim Merlins,Oldenburg,168.5,1.91,0.8344,Over
8,16:00,Alemanha Bbl,Wurzburg,Chemnitz,157.5,1.91,0.8436,Over
9,06:00,Austrália Nbl,Perth Wildcats,Adelaide 36ers,172.5,1.91,0.8369,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
14,08:35,China Cba,Tianjin,Shanxi Zhongyu,-1.5,1.87,0.5883,AH Zebra
15,08:35,China Cba,Zhejiang Guangsha,Shandong,-8.5,1.87,0.5408,AH Zebra
39,22:00,Eua Nba,Denver Nuggets,Chicago Bulls,-9.5,1.91,0.5363,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,16:45,Espanha Acb,Tenerife,MoraBanc Andorra,-7.5,1.87,0.6348,AH Zebra
4,21:00,Eua Nba,Minnesota Timberwolves,Utah Jazz,-7.5,1.91,0.5011,AH Zebra
13,08:35,China Cba,Shenzhen,Guangzhou,-8.5,1.87,0.5547,AH Zebra
15,08:35,China Cba,Zhejiang Guangsha,Shandong,-8.5,1.87,0.5107,AH Zebra
17,12:00,França Lnb,JL Bourg,Roanne,-10.5,1.91,0.6317,AH Zebra
18,14:30,França Lnb,Chalon/Saone,Boulogne-Levallois,-9.5,1.91,0.6330,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
4,21:00,Eua Nba,Minnesota Timberwolves,Utah Jazz,-7.5,1.91,0.5022,AH Zebra
15,08:35,China Cba,Zhejiang Guangsha,Shandong,-8.5,1.87,0.5166,AH Zebra


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,16:45,Espanha Acb,Tenerife,MoraBanc Andorra,-7.5,1.87,0.5883,AH Zebra
4,21:00,Eua Nba,Minnesota Timberwolves,Utah Jazz,-7.5,1.91,0.5021,AH Zebra
13,08:35,China Cba,Shenzhen,Guangzhou,-8.5,1.87,0.5003,AH Zebra
15,08:35,China Cba,Zhejiang Guangsha,Shandong,-8.5,1.87,0.5003,AH Zebra
17,12:00,França Lnb,JL Bourg,Roanne,-10.5,1.91,0.5850,AH Zebra
18,14:30,França Lnb,Chalon/Saone,Boulogne-Levallois,-9.5,1.91,0.5893,AH Zebra
